In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from matplotlib import pyplot as plt 
%matplotlib inline
import seaborn as sns
#
from sklearn.metrics import accuracy_score
#

In [ ]:
big_data_train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', nrows=100000)
questions_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
lectures_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv')
test_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_test.csv')
test_df.head()

In [ ]:
X = big_data_train[['user_id', 'content_id', 'content_type_id', 'task_container_id', 'prior_question_elapsed_time', 'prior_question_had_explanation']]
y = big_data_train[['answered_correctly']]
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
#
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from sklearn.experimental import (enable_iterative_imputer,)
from sklearn import impute
#
num_cols = ["user_id", "content_id", "content_type_id", "task_container_id", "prior_question_elapsed_time", "prior_question_had_explanation"]
imputer = impute.IterativeImputer()
imputed = imputer.fit_transform(X_train[num_cols])
X_train.loc[:, num_cols] = imputed
#X_train
imputed = imputer.fit_transform(X_test[num_cols])
X_test.loc[:, num_cols] = imputed
X_test

In [ ]:
#
from sklearn.ensemble import RandomForestClassifier
#
rfclf = RandomForestClassifier()
rfclf.fit(X_train,y_train)

In [ ]:
y_pred=rfclf.predict(X_train)
t_pred=rfclf.predict(X_test)
print(accuracy_score(y_pred, y_train))
print(accuracy_score(t_pred, y_test))

In [ ]:
#params
from sklearn.model_selection import cross_validate, GridSearchCV
#
params = {
    'max_features':[0.4, "auto"],
    'n_estimators':[15, 200],
    'min_samples_leaf':[1, 0.1],
    'random_state':[10, 42, 100]
}
#
cv = GridSearchCV(
    rfclf,
    params,
    n_jobs = -1
).fit(X_train,y_train)
cv.best_params_

In [ ]:
rfc_m = RandomForestClassifier(
    **{'max_features': 0.4,
       'min_samples_leaf': 0.1,
       'n_estimators': 15,
       'random_state': 10,
      }
    )
rfc_m.fit(X_train,y_train)

In [ ]:
y_pred=rfc_m.predict(X_train)
t_pred=rfc_m.predict(X_test)
print(accuracy_score(y_pred, y_train))
print(accuracy_score(t_pred, y_test))

In [ ]:
import xgboost as xgb
#
xgb_m = xgb.XGBClassifier()
#xgbclf = xgb.XGBClassifier(random_state=42)
#xgbclf.fit(X_train,y_train, early_stopping_rounds=10, eval_set=[(X_test, y_test)],)
xgb_m.fit(X_train,y_train)

In [ ]:
y_pred=xgb_m.predict(X_train)
t_pred=xgb_m.predict(X_test)
print(accuracy_score(y_pred, y_train))
print(accuracy_score(t_pred, y_test))

In [ ]:
#
for col, val in sorted(
    zip(
        X.columns,
        xgb_m.feature_importances_,
    ),
    key=lambda x: x[1],
    reverse=True,
)[:5]:
    print(f"{col:10}{val:10.3f}")

In [ ]:
#
fig, ax = plt.subplots(figsize=(6, 4))
xgb.plot_importance(xgb_m, ax=ax)

In [ ]:
#
booster = xgb_m.get_booster()
print(booster.get_dump()[0])

In [ ]:
from sklearn.neural_network import MLPClassifier
#
mlp_m = MLPClassifier()
mlp_m.fit(X_train,y_train)

In [ ]:
y_pred=mlp_m.predict(X_train)
t_pred=mlp_m.predict(X_test)
print(accuracy_score(y_pred, y_train))
print(accuracy_score(t_pred, y_test))

In [ ]:
#lgbm
import lightgbm as lgb

In [ ]:
#
lgb_m = lgb.LGBMClassifier(random_state=42)
#lgb_m = lgb.LGBMClassifier()
lgb_m.fit(X_train,y_train)

In [ ]:
y_pred=lgb_m.predict(X_train)
t_pred=lgb_m.predict(X_test)
print(accuracy_score(y_pred, y_train))
print(accuracy_score(t_pred, y_test))

In [ ]:
#
for col, val in sorted(
    zip(
        X.columns,
        lgb_m.feature_importances_,
    ),
    key=lambda x: x[1],
    reverse=True,
)[:5]:
    print(f"{col:10}{val:10.3f}")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
lgb.plot_importance(lgb_m, ax=ax)
fig.tight_layout()